# A DL model with Keras

In this notebook:
* How to load a CSV dataset ready for use with Keras.
* How to define and compile a Multilayer Perceptron model in Keras.   How to evaluate a Keras model on a validation dataset.

## The steps

There is not a lot of code required, but we are going to step over it slowly so that you will know how to create your own models in the future. The steps you are going to cover in this tutorial are as follows:
1. Load Data.
2. Define Model.
3. Compile Model.
4. Fit Model.
5. Evaluate Model.
6. Tie It All Together.

## The dataset

We will use the Pima Indians Onset of Diabetes Dataset.

Given that all attributes are numerical, this makes it easy to use directly with NN that expect numerical inputs and output values, and ideal for our first NN in Keras. 

Now we can revela how good you can get here!

The baseline accuracy if all predictions are made as no onset of diabetes is 65.1%. Top results on the dataset are in the range of 77.7% accuracy using 10-fold cross-validation ([here](http://www.is.umk.pl/projects/datasets.html#Diabetes)). 

## Load Data

Whenever we work with machine learning algorithms that use a stochastic process (e.g. random numbers), it is a good idea to initialize the random number generator with a fixed seed value. This is so that you can run the same code again and again and get the same result. This is useful if you need to demonstrate a result, compare algorithms using the same source of randomness or to debug a part of your code. You can initialize the random number generator with any seed you like, for example:

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


Now we can load our Pima Indians dataset. You can now load the file directly using the NumPy function `loadtxt()`. There are eight input variables and one output variable (the last column). Once loaded we can split the dataset into input variables (X) and the output class variable (Y ).

In [2]:
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.data.csv", delimiter=",")
# split into input and output variables
X = dataset[:,0:8]
Y = dataset[:,8]

We are now ready to define our neural network model.

## Define Model

Models in Keras are defined as a sequence of layers. **We create a Sequential model and add layers one at a time until we are happy with our network topology**. The first thing to get right is to ensure the input layer has the right number of inputs. This can be specified when creating the first layer with the input dim argument and setting it to 8 for the 8 input variables.

**How do we know the number of layers to use and their types?** This is a very hard question. There are heuristics that we can use but most often the best network structure is found through a process of trial and error experimentation. Generally, _you need a network large enough to capture the structure of the problem_ if that helps at all. In this example we will use _a fully-connected network structure with three layers_.

**Fully connected layers are defined using the Dense class**:
* We can specify the number of neurons in the layer as the first argument
* we can specify the activation function using the activation argument.
* We will use the rectifier (relu) activation function on the first two layers and the sigmoid activation function in the output layer.
* We use a sigmoid activation function on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5. 

Why relu? It used to be the case that sigmoid and tanh activation functions were preferred for all layers. These days, better performance is seen using the relu. 

We can piece it all together by adding each layer. The first hidden layer has 12 neurons and expects 8 input variables (e.g. input dim=8). The second hidden layer has 8 neurons and finally the output layer has 1 neuron to predict the class (onset of diabetes or not).

In [4]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model

The figure below provides a depiction of the network structure.

<img src="images/NNstructure.png" style="width:200px"/>

## Compile Model

Now that the model is defined, we can compile it. 

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. **The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware**.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to make predictions for this problem.

We must specify the **loss function** to use to evaluate a set of weights, the **optimizer** used to search through different weights for the network and **any optional metrics** we would like to collect and report during training. 

In this case we will use **logarithmic loss**, which for a binary classification problem is defined in Keras as the **binary crossentropy**. 

We will also use the efficient **gradient descent algorithm Adam** for no other reason that it is an efficient default. Learn more about the Adam optimization algorithm in the paper [Adam: A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980). 

Finally, because it is a classification problem, we will collect and report the classification **accuracy** as the metric.

In [5]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit Model

We have defined our model and compiled it ready for efficient computation. Now it is time to execute the model on some data. We can train or fit our model on our loaded data by calling the $fit()$ function on the model.

The training process will run for a fixed number of iterations through the dataset called **epochs**, that we must specify using the epochs argument. 

We can also set the number of instances that are evaluated before a weight update in the network is performed called the **batch size** and set it using the $batch size$ argument. 

For this problem we will run for a small number of epochs (150) and use a relatively small batch size of 10. Again, these can be chosen experimentally by trial and error.

_NOTE: This is where the work happens on your CPU or GPU._

In [6]:
# Fit the model
model.fit(X, Y, epochs=150, batch_size=10)

Epoch 1/150
768/768 [==============================] - 0s - loss: 5.2286 - acc: 0.4518      
Epoch 2/150
768/768 [==============================] - 0s - loss: 1.7954 - acc: 0.5521     

## Evaluate Model

We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset. This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for the training and evaluation of your model.

You can evaluate your model on your training dataset using the evaluation() function on your model and pass it the same input and output used to train the model. This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

In [11]:
# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/768 [>.............................] - ETA: 0s
acc: 77.73%


## Tie It All Together

You have just seen how you can easily create your first neural network model in Keras. Let’s tie it all together into a complete code example.

In [13]:
# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.data.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Fit the model
model.fit(X, Y, epochs=150, batch_size=10)
# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/150
768/768 [==============================] - 0s - loss: 3.7417 - acc: 0.5990      
Epoch 2/150
768/768 [==============================] - 0s - loss: 0.9420 - acc: 0.5911     

Running this example, you should see a message for each of the 150 epochs printing the loss and accuracy for each, followed by the final evaluation of the trained model on the training dataset. It takes about 10 seconds to execute on my workstation running on the CPU with a TF backend.

## Summary

What we learned:

* we discovered how to create your first neural network model using the powerful Keras Python library for deep learning. 
* Specifically you learned the five key steps in using Keras to create a neural network or deep learning model, step-by-step including:

    1. How to load data.
    2. How to define a neural network model in Keras.
    3. How to compile a Keras model using the efficient numerical backend.   How to train a model on data.
    4. How to evaluate a model on data.

## What's next 

Discover different ways that you can evaluate your models and estimate their performance on unseen data.